## LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?") // text-davinci-003 2024.01.04 out of service -> 실행 안 됨
b = chat.predict("How many planets are there?") # In service gpt-3.5-turbo

b

'As of August 2021, there are eight recognized planets in our solar system. They are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that in 2006, the International Astronomical Union (IAU) redefined the definition of a planet, leading to the reclassification of Pluto as a "dwarf planet."'

## Predict Messages

In [35]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1 # temperature가 높을수록 창의력이 떨어짐, 답변의 활용도가 낮음
)

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content = "Ciao, mi chiam Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)


AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

## prompt

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

## chatPromptTemplate : message로부터 template을 만듬
## PromptTemplate : String을 이용해서 template을 만듬
## template에는 검증계층(validation layer)이 있음 -> 사용자가 볼 수 없는 곳에서 template이 질문에 대한 변수를 정의함.

template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a = 'Mexico', country_b="Thailand")
## template.format은 질문을 정리해서 보여줌


print(type(prompt))

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)



<class 'str'>


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

## FewShoutPromptTemplate


In [38]:
# TODO: langchain modules study

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate # 질문을 받고 답변을 할 수 있는 PROMPT를 제작하는 모듈
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate # AI가 답변하는 형식을 학습시킬 수 있는 모듈 -> 대화 방식을 데이터 베이스에 저장하여 가지고 오면 편함
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

t = PromptTemplate(
    template="what is the capital of {country}",
    input_variables=['country'],
)

t.format(country="France")

'what is the capital of France'

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
{
    "question": "what do you know about Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "what do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "what do you kow about France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]
# chat.predict("what do you know about France?")

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="Human: What do you know about{country}?",
    input_variables=['country']
)
                                              

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Turkey"
})


AI
    I know this:
    Capital: Ankara
    Language: Turkish
    Food: Kebab and Baklava
    Currency: Turkish Lira

AIMessageChunk(content='AI\n    I know this:\n    Capital: Ankara\n    Language: Turkish\n    Food: Kebab and Baklava\n    Currency: Turkish Lira')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    model = "gpt-3.5-turbo",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?")
    ]
)


final_prompt.format(country = "Japan")

chain = final_prompt | chat

response = chain.invoke(
    {"country" : "Turkey"}
)

print(response)

## 결제????


## LengthBasedExampleSelector

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 예제들을 형식화함, 예제의 양이 얼마나 되는지 확인할 수 있음


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,

    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt = example_prompt,
    max_length = 300,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country = "Brazil")


'Human: Italy?\nAI\n    Here is what I know:\n    Capital : Rome\n    Language: Italian\n    Good: Pizza and Pasta\n    Currency: Euro\n\nHuman: Greece?\nAI\n    I know this:\n    Capital: Athens\n    Language: Greek\n    Good:Souvlaki and Feta Cheese\n    Currency: Euro\n\nHuman: France?\nAII know this:\n    Capital : Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n\nHuman: What do you know about Brazil?'

## BaseExampleSelector

In [ ]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 예제들을 형식화함, 예제의 양이 얼마나 되는지 확인할 수 있음
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country = "Brazil")


## Serialization and Composition
#### 여러 개의 prompt를 만들어서 병합시키는 작업이 필요함.

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
# prompt 파일을 json 또는 yaml 으로 만들어서 import 시킴

prompt = load_prompt("./prompt.yaml")

prompt.format(country = "korea")

'what is the capital of korea'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)
intro = PromptTemplate